In [7]:
# change print to string
# copy template
from pathlib import Path
from data_loading import load_annotated_TFiso1_collection

shared_dir = Path('/Users/lukelambourne/Dropbox (Partners HealthCare)/TF_isoforms')
tf_webpage_dir = shared_dir / 'website'

with open('molstar_template.js', 'r') as f:
    template = f.read()

tfs = load_annotated_TFiso1_collection()
for gene in tfs.values():
    exon_colors = gene._get_exon_colors()
    iso_json = '['
    for isoform in gene.cloned_isoforms:
        exon_pos = 1
        nt_diff = 0
        iso_json += '\n{name: \'' + isoform.name + '\','
        exon_boundaries_in_aa = []
        exon_colors_for_iso = []
        for exon in isoform.exons:
            n_nt_to_exon = (exon.end - exon.start) + nt_diff
            n_aa_exon = round(n_nt_to_exon / 3)
            nt_diff = {0: 0, 1: 1, 2: -1}[n_nt_to_exon % 3]
            color = exon_colors[(isoform.name, exon.start, exon.end)]
            start = exon_pos
            end = (exon_pos + n_aa_exon) - 1
            exon_boundaries_in_aa.append([start, end])
            exon_colors_for_iso.append(color)
            exon_pos += n_aa_exon
        iso_json += '\nexon_boundaries: ' + str(exon_boundaries_in_aa) + ','
        iso_json += '\nexon_colors: [' + ', '.join([f'{{r: {int(r * 255)}, g: {int(g * 255)}, b: {int(b * 255)}}}' for r, g, b in exon_colors_for_iso]) + ']'
        iso_json += '},'
    iso_json += ']'
    with open(tf_webpage_dir / 'pages/{}-molstar.js'.format(gene.name), 'w') as f:
        f.write(template.replace('cloned_isoform_json_array', iso_json))

reading from cache
